<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_Parte4_Cpp_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

Se realizó una comparación de diferentes métodos de sincronización al realizar la suma. Particularmente std::atomic y std::mutex, sumado a una validación de resultados con std::reduce y un for común. 

Para realizar la prueba generamos un vector con 500k elementos aleatorios. 

A demás se agregó una comparación de tiempos de ejecución de cada método.

Por los resultados obtenidos determinamos que el método más veloz es std::mutex y el más lento std::atomic. Obviamente esto no es determinista y sería interesante investigar en mayor profundidad

In [15]:
%%writefile main.cpp
#include <iostream>
#include <vector>
#include <string>
#include <thread>
#include <atomic>
#include <mutex>
#include <algorithm>
#include <numeric>
#include <random>
#include <functional>
#include <chrono>
#include <type_traits>

#define THREADS_NUMBER 2
#define MAX_ELEMENT_QUANTITY 500'000
#define MIN_VALUE -1000
#define MAX_VALUE 1000

/** Busca el mejor clock disponible. */
using clock_type = typename std::conditional<
  std::chrono::high_resolution_clock::is_steady,
  std::chrono::high_resolution_clock,
  std::chrono::steady_clock>::type;


void AddElements(std::vector<int>& vec);
int GetTotalSum(const std::vector<int>& vec);
int GetTotalSumReduce(const std::vector<int>& vec);
int GetTotalSumAtomic(const std::vector<int>& vec);
int GetTotalSumMutex(const std::vector<int>& vec);
void SumVectorAtomic(const std::vector<int>& vec, int form, int to, std::atomic<int>& sum);
void SumVectorMutex(const std::vector<int>& vec, int from, int to, int& sum, std::mutex& m);
void PrintResult(std::function<int(std::vector<int>&)> fun, std::vector<int>& vec, std::string s);

int main(int argc, char* argv[])
{
    std::vector<int> vec;
    AddElements(vec);

    PrintResult(GetTotalSum, vec, "Total suma for:    ");
    PrintResult(GetTotalSumReduce, vec, "Total suma reduce: ");
    PrintResult(GetTotalSumAtomic, vec, "Total suma atomic: ");
    PrintResult(GetTotalSumMutex, vec, "Total suma mutex:  ");

    return EXIT_SUCCESS;
}

int GetTotalSum(const std::vector<int>& vec)
{
    int sum = 0;
    for(int i = 0; i < vec.size(); ++i)
        sum+= vec[i];
    return sum;
}

int GetTotalSumReduce(const std::vector<int>& vec)
{
    return std::reduce(vec.begin(), vec.end());
}

int GetTotalSumAtomic(const std::vector<int>& vec)
{
    std::atomic<int> sum = 0;
    std::vector<std::thread> workers;
    int mid_vec = vec.size()/2;

    workers.push_back(std::thread(SumVectorAtomic, std::cref(vec), 0, mid_vec, std::ref(sum)));
    workers.push_back(std::thread(SumVectorAtomic, std::cref(vec), mid_vec, vec.size(), std::ref(sum)));
    
    for(auto& worker : workers)
    {
        worker.join();
    }

    return sum;
}

void SumVectorAtomic(const std::vector<int>& vec, int from, int to, std::atomic<int>& sum)
{
    for (int i = from; i < to; ++i)
        sum += vec[i];
}

int GetTotalSumMutex(const std::vector<int>& vec)
{
    int sum = 0;
    std::vector<std::thread> workers;
    int mid_vec = vec.size()/2;
    std::mutex mutex;

    workers.push_back(std::thread(SumVectorMutex, std::cref(vec), 0, mid_vec, std::ref(sum),std::ref(mutex)));
    workers.push_back(std::thread(SumVectorMutex, std::cref(vec), mid_vec, vec.size(), std::ref(sum), std::ref(mutex)));
    
    for(auto& worker : workers)
    {
        worker.join();
    }

    return sum;
}

void SumVectorMutex(const std::vector<int>& vec, int from, int to, int& sum, std::mutex& m)
{
    std::lock_guard<std::mutex> guard(m);
    for (int i = from; i < to; ++i)
        sum += vec[i];
}

void AddElements(std::vector<int>& vec)
{
    std::random_device rd; // Obtenemos un número random del hardware
    std::mt19937 gen(rd()); // Generamos la semilla (seed)
    std::uniform_int_distribution<> distr(MIN_VALUE, MAX_VALUE); // Definimos el rango

    for(int i=0; i < MAX_ELEMENT_QUANTITY; ++i)
        vec.push_back(distr(gen));
}

void PrintResult(std::function<int(std::vector<int>&)> fun, std::vector<int>& vec, std::string s)
{
    auto t_start = clock_type::now();
    int result = fun(vec);
    auto t_end = clock_type::now();
    std::cout << s << result 
        << " Tiempo: " << std::chrono::duration<double, std::milli>(t_end-t_start).count()
        << "ms" << std::endl;
}


Overwriting main.cpp


In [16]:
!g++ -pthread -std=c++17 main.cpp -o main && ./main

Total suma for:    41211 Tiempo: 2.39464ms
Total suma reduce: 41211 Tiempo: 4.89041ms
Total suma atomic: 41211 Tiempo: 3.89855ms
Total suma mutex:  41211 Tiempo: 2.23755ms
